# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información

El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión, recall, F1-score, Mean Average Precision (MAP) y Normalized Discounted Cumulative Gain (nDCG)*.

Seguirás los siguientes pasos:

 ### Proporcionar un Conjunto de Datos:

- **Corpus de Documentos:** Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.

In [1]:
#Librerias
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
file_path = '/kaggle/input/03ranking-corpus/03ranking_corpus.xml'
tree = ET.parse(file_path)
root = tree.getroot()

# Crear una lista para almacenar los documentos con sus metadatos
documents = []

# Iterar a través de cada elemento 'document' en el archivo XML
for doc in root.findall('document'):
    doc_id = doc.get('id')
    title = doc.find('title').text
    keywords = doc.find('keywords').text
    author = doc.find('author').text
    date = doc.find('date').text
    
    documents.append({
        'id': doc_id,
        'title': title,
        'keywords': keywords,
        'author': author,
        'date': date
    })

In [3]:
# Mostrar el resultado
for document in documents:
    print(f"ID: {document['id']}")
    print(f"Title: {document['title']}")
    print(f"Keywords: {document['keywords']}")
    print(f"Author: {document['author']}")
    print(f"Date: {document['date']}")
    print("-" * 60)

ID: 1
Title: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
Keywords: telemedicina, salud crónica, tratamiento, tecnología médica
Author: Dr. Juan Pérez
Date: 2023-01-15
------------------------------------------------------------
ID: 2
Title: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
Keywords: nutrición, rendimiento académico, salud mental, estudiantes
Author: Dra. María López
Date: 2023-02-10
------------------------------------------------------------
ID: 3
Title: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.
Keywords: amistad, bienestar estudiantil, campus, relaciones sociales
Author: Miguel Rodríguez
Date: 2023-03-05
------------------------------------------------------------
ID: 4
Title: El rol de las bibliotecas universitarias en el fomento de la investigación académica.
Keywords: bibliotecas universitarias, investigación, academia, 

- **Consultas:** Define un conjunto de consultas específicas.

In [4]:
# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

- **Juicios de Relevancia:** Proporciona una lista de qué documentos son relevantes para cada consulta.

In [5]:
# Función para procesar el texto y extraer palabras clave
def process_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Reemplazar caracteres no alfanuméricos por espacios
    import re
    text = re.sub(r'[^a-záéíóúñü]+', ' ', text)
    # Tokenizar y eliminar palabras vacías si es necesario
    tokens = text.strip().split()
    return set(tokens)

In [6]:
# Mostrar las palabras clave extraídas
for query_id, query_text in queries.items():
    # Procesar el texto de la consulta
    keywords = process_text(query_text)
    print(f"Consulta {query_id}: {keywords}")

Consulta 1: {'impacto', 'salud', 'estudiantes', 'académico', 'de', 'el', 'mental', 'en', 'la', 'universitarios', 'rendimiento', 'los'}
Consulta 2: {'campus', 'el', 'bienestar', 'emocional', 'y', 'en', 'universitario', 'extracurriculares', 'actividades'}
Consulta 3: {'estudiantes', 'universitarias', 'para', 'estrés', 'reducir', 'el', 'en', 'estrategias'}


In [7]:
# Función de similitud coseno
def cosine_similarity(query, doc):
    query_tokens = process_text(query)
    doc_tokens = process_text(doc)

    # Crear el vocabulario (todas las palabras únicas en la consulta y el documento)
    vocabulary = list(query_tokens | doc_tokens)  # Unión de los conjuntos

    # Crear los vectores de frecuencia (binarios)
    query_vector = [1 if word in query_tokens else 0 for word in vocabulary]
    doc_vector = [1 if word in doc_tokens else 0 for word in vocabulary]

    # Calcular el producto punto y las normas
    dot_product = np.dot(query_vector, doc_vector)
    norm_query = np.linalg.norm(query_vector)
    norm_doc = np.linalg.norm(doc_vector)

    # Similitud coseno
    if norm_query == 0 or norm_doc == 0:
        return 0  # No hay similitud si uno de los vectores es cero
    return dot_product / (norm_query * norm_doc)

In [8]:
# Función de similitud de Jaccard
def jaccard_similarity(query, doc):
    query_tokens = process_text(query)
    doc_tokens = process_text(doc)

    # Intersección y unión de los conjuntos
    intersection = query_tokens & doc_tokens
    union = query_tokens | doc_tokens
    return len(intersection) / len(union)

In [9]:
# Uso de la similaridad del coseno
for query_id, query_text in queries.items():
    print(f"Consulta {query_id}: {query_text}")
    
    for doc in documents:
        doc_text = doc['title'] + ' ' + doc['keywords'] 
        cosine_score = cosine_similarity(query_text, doc_text) 
        
        print(f"  Documento {doc['id']}: {doc['title']}")
        print(f"    Similitud Coseno: {cosine_score:.4f}")
    print("\n")

Consulta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
  Documento 1: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
    Similitud Coseno: 0.3203
  Documento 2: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
    Similitud Coseno: 0.6405
  Documento 3: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.
    Similitud Coseno: 0.3501
  Documento 4: El rol de las bibliotecas universitarias en el fomento de la investigación académica.
    Similitud Coseno: 0.3203
  Documento 5: Cómo los espacios verdes en los campus universitarios pueden mejorar la concentración y reducir el estrés.
    Similitud Coseno: 0.3608
  Documento 6: La influencia de la cultura universitaria en los hábitos saludables y el bienestar de los estudiantes.
    Similitud Coseno: 0.4804
  Documento 7: La importancia del sueño en la salud menta

In [10]:
# Uso de la similaridad de Jaccard
for query_id, query_text in queries.items():
    print(f"Consulta {query_id}: {query_text}")
    
    for doc in documents:
        doc_text = doc['title'] + ' ' + doc['keywords'] 
        jaccard_score = jaccard_similarity(query_text, doc_text) # Aplicamos el método para calcular la similaridad de Jaccard
        
        print(f"  Documento {doc['id']}: {doc['title']}")
        print(f"    Similitud de Jaccard: {jaccard_score:.4f}")
    print("\n")

Consulta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
  Documento 1: El aumento de la telemedicina para el tratamiento de condiciones de salud crónicas.
    Similitud de Jaccard: 0.1905
  Documento 2: Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
    Similitud de Jaccard: 0.4706
  Documento 3: Estudio sobre cómo las relaciones de amistad contribuyen al bienestar de los estudiantes en el campus.
    Similitud de Jaccard: 0.2083
  Documento 4: El rol de las bibliotecas universitarias en el fomento de la investigación académica.
    Similitud de Jaccard: 0.1905
  Documento 5: Cómo los espacios verdes en los campus universitarios pueden mejorar la concentración y reducir el estrés.
    Similitud de Jaccard: 0.2174
  Documento 6: La influencia de la cultura universitaria en los hábitos saludables y el bienestar de los estudiantes.
    Similitud de Jaccard: 0.3158
  Documento 7: La importancia del

 ### Calcular Resultados de Búsqueda:

- Obten los resultados ordenados de dos sistemas de recuperación para cada consulta.

In [19]:
# Número de documentos a mostrar
k = 10

# Función para rankear los documentos
def rank_documents(queries, similarity_function):
    rankings = {}
    
    for query_id, query_text in queries.items():
        similarity_scores = []
        
        # Calcular la similitud de cada documento con la consulta
        for doc in documents:
            doc_text = doc['title'] + ' ' + doc['keywords']
            similarity_score = similarity_function(query_text, doc_text)
            similarity_scores.append((doc['id'], doc['title'], similarity_score))
        
        # Ordenar los documentos por la similitud (de mayor a menor)
        ranked_docs = sorted(similarity_scores, key=lambda x: x[2], reverse=True)
        
        # Guardar los k primeros documentos
        rankings[query_id] = ranked_docs[:k]
    
    return rankings

In [20]:
# Rankear los documentos usando la similitud coseno
cosine_rankings = rank_documents(queries, cosine_similarity)

# Mostrar los rankings
print("Ranking por Similitud Coseno:")
for query_id, ranked_docs in cosine_rankings.items():
    print(f"Consulta {query_id}: {query_text}")
    for rank, (doc_id, doc_title, score) in enumerate(ranked_docs, start=1):
        print(f" {rank}. Documento {doc_id} - (Similitud Coseno: {score:.4f})")
        print(f" {doc_title}")
    print("\n")

Ranking por Similitud Coseno:
Consulta 1: Estrategias universitarias para reducir el estrés en estudiantes
 1. Documento 2 - (Similitud Coseno: 0.6405)
 Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
 2. Documento 7 - (Similitud Coseno: 0.6405)
 La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
 3. Documento 14 - (Similitud Coseno: 0.6301)
 Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.
 4. Documento 11 - (Similitud Coseno: 0.6172)
 Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.
 5. Documento 13 - (Similitud Coseno: 0.5809)
 Estrategias para mejorar la salud mental en estudiantes universitarios: el rol del apoyo psicológico y los grupos de estudio en el campus.
 6. Documento 8 - (Similitud Coseno: 0.5217)
 La influencia del apoyo familiar y social en el bienestar emocional de 

In [21]:
# Rankear los documentos usando la similitud de Jaccard
jaccard_rankings = rank_documents(queries, jaccard_similarity)

# Mostrar los rankings
print("Ranking por Similitud de Jaccard:")
for query_id, ranked_docs in jaccard_rankings.items():
    print(f"Consulta {query_id}: {query_text}")
    for rank, (doc_id, doc_title, score) in enumerate(ranked_docs, start=1):
        print(f" {rank}. Documento {doc_id} - (Similitud Jaccard: {score:.4f})")
        print(f" {doc_title}")
    print("\n")

Ranking por Similitud de Jaccard:
Consulta 1: Estrategias universitarias para reducir el estrés en estudiantes
 1. Documento 2 - (Similitud Jaccard: 0.4706)
 Cómo la nutrición balanceada afecta el rendimiento académico y la salud mental en estudiantes.
 2. Documento 7 - (Similitud Jaccard: 0.4706)
 La importancia del sueño en la salud mental y el rendimiento académico en jóvenes universitarios.
 3. Documento 14 - (Similitud Jaccard: 0.4500)
 Cómo el acceso a servicios de salud mental en la universidad puede mejorar el desempeño académico.
 4. Documento 11 - (Similitud Jaccard: 0.4444)
 Impacto de la práctica regular de ejercicio en la reducción del estrés académico en estudiantes universitarios.
 5. Documento 13 - (Similitud Jaccard: 0.3913)
 Estrategias para mejorar la salud mental en estudiantes universitarios: el rol del apoyo psicológico y los grupos de estudio en el campus.
 6. Documento 8 - (Similitud Jaccard: 0.3500)
 La influencia del apoyo familiar y social en el bienestar emo

 ### Calcular las Métricas de Evaluación:

- Calcular las siguientes métricas para cada sistema y consulta:
- 
  - Precisión en el top-k (Prec@k)
  - 
Recal
  - 
F1-sco
  - e
Mean Average Precision (M
  - P)
nDCG

In [23]:
# Función para determinar si un documento es relevante según el umbral
def is_relevant(score, threshold):
    return score >= threshold

# Función para calcular la precisión en el top-k
def precision_at_k(ranked_scores, k):
    relevant_docs = sum([1 for score in ranked_scores[:k] if score[1]])
    return relevant_docs / k

# Función para calcular el recall
def recall(ranked_scores, total_relevant_docs):
    relevant_retrieved = sum([1 for score in ranked_scores if score[1]])
    return relevant_retrieved / total_relevant_docs if total_relevant_docs > 0 else 0

# Función para calcular F1-score
def f1_score(precision, recall):
    return (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Función para calcular Mean Average Precision (MAP)
def mean_average_precision(ranked_scores):
    relevant_docs = 0
    sum_precisions = 0
    for i, (score, is_rel) in enumerate(ranked_scores):
        if is_rel:
            relevant_docs += 1
            sum_precisions += relevant_docs / (i + 1)
    return sum_precisions / relevant_docs if relevant_docs > 0 else 0

# Función para calcular nDCG
def ndcg(ranked_scores, k):
    dcg = sum([score[1] / np.log2(i + 2) for i, score in enumerate(ranked_scores[:k])])
    ideal_scores = sorted([score[1] for score in ranked_scores], reverse=True)[:k]
    idcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(ideal_scores)])
    return dcg / idcg if idcg > 0 else 0



In [40]:
# Definimos el umbral de relevancia mínimo
relevance_threshold_cos = 0.47 # Valor dado luego del análisis del ranking
relevance_threshold_jac = 0.29 # Valor dado luego del análisis del ranking

# Función para obtener las métricas para cada sistema y consulta
def evaluate_similarity_cos(queries, documents, similarity_func):
    results = {}
    for query_id, query_text in queries.items():
        ranked_docs = []
        
        # Calcular la similitud para cada documento
        for doc in documents:
            doc_text = doc['title'] + ' ' + doc['keywords']
            score = similarity_func(query_text, doc_text)
            is_rel = is_relevant(score, relevance_threshold_cos)
            ranked_docs.append((score, is_rel))
        
        # Ordenar documentos por similitud en orden descendente
        ranked_docs.sort(reverse=True, key=lambda x: x[0])
        
        # Calcular métricas de evaluación
        total_relevant_docs = sum([is_rel for _, is_rel in ranked_docs])  # Número de documentos relevantes
        prec_k = precision_at_k(ranked_docs, k)
        rec = recall(ranked_docs, total_relevant_docs)
        f1 = f1_score(prec_k, rec)
        map_score = mean_average_precision(ranked_docs)
        ndcg_score = ndcg(ranked_docs, k)
        
        # Guardar resultados
        results[query_id] = {
            'Prec@k': prec_k,
            'Recall': rec,
            'F1-score': f1,
            'MAP': map_score,
            'nDCG': ndcg_score
        }
    
    return results

def evaluate_similarity_jac(queries, documents, similarity_func):
    results = {}
    for query_id, query_text in queries.items():
        ranked_docs = []
        
        # Calcular la similitud para cada documento
        for doc in documents:
            doc_text = doc['title'] + ' ' + doc['keywords']
            score = similarity_func(query_text, doc_text)
            is_rel = is_relevant(score, relevance_threshold_jac)
            ranked_docs.append((score, is_rel))
        
        # Ordenar documentos por similitud en orden descendente
        ranked_docs.sort(reverse=True, key=lambda x: x[0])
        
        # Calcular métricas de evaluación
        total_relevant_docs = sum([is_rel for _, is_rel in ranked_docs])  # Número de documentos relevantes
        prec_k = precision_at_k(ranked_docs, k)
        rec = recall(ranked_docs, total_relevant_docs)
        f1 = f1_score(prec_k, rec)
        map_score = mean_average_precision(ranked_docs)
        ndcg_score = ndcg(ranked_docs, k)
        
        # Guardar resultados
        results[query_id] = {
            'Prec@k': prec_k,
            'Recall': rec,
            'F1-score': f1,
            'MAP': map_score,
            'nDCG': ndcg_score
        }
    
    return results

 ### Análisis y Comparación:

- Comparar los resultados de los dos sistemas utilizando las métricas calculadas.

In [46]:
# Evaluación y comparación de resultados
cosine_results = evaluate_similarity_cos(queries, documents, cosine_similarity)

# Mostrar resultados
for query_id in queries:
    print(f"\nConsulta {query_id}: {queries[query_id]}")
    print("Métricas para la Similitud Coseno:")
    print(f"  Precisión en el top-k Prec@{k}: {cosine_results[query_id]['Prec@k']:.4f}")
    print(f"  Recall: {cosine_results[query_id]['Recall']:.4f}")
    print(f"  F1-score: {cosine_results[query_id]['F1-score']:.4f}")
    print(f"  Mean Average Precision MAP: {cosine_results[query_id]['MAP']:.4f}")
    print(f"  nDCG@{k}: {cosine_results[query_id]['nDCG']:.4f}")


Consulta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
Métricas para la Similitud Coseno:
  Precisión en el top-k Prec@10: 0.9000
  Recall: 1.0000
  F1-score: 0.9474
  Mean Average Precision MAP: 1.0000
  nDCG@10: 1.0000

Consulta 2: Actividades extracurriculares y bienestar emocional en el campus universitario
Métricas para la Similitud Coseno:
  Precisión en el top-k Prec@10: 0.1000
  Recall: 1.0000
  F1-score: 0.1818
  Mean Average Precision MAP: 1.0000
  nDCG@10: 1.0000

Consulta 3: Estrategias universitarias para reducir el estrés en estudiantes
Métricas para la Similitud Coseno:
  Precisión en el top-k Prec@10: 0.1000
  Recall: 1.0000
  F1-score: 0.1818
  Mean Average Precision MAP: 1.0000
  nDCG@10: 1.0000


In [47]:
# Mostrar resultados
jaccard_results = evaluate_similarity_jac(queries, documents, jaccard_similarity)

# Mostrar resultados
for query_id in queries:
    print(f"\nConsulta {query_id}: {queries[query_id]}")
    print("Métricas para la Similitud Jaccard:")
    print(f"  Precisión en el top-k Prec@{k}: {jaccard_results[query_id]['Prec@k']:.4f}")
    print(f"  Recall: {jaccard_results[query_id]['Recall']:.4f}")
    print(f"  F1-score: {jaccard_results[query_id]['F1-score']:.4f}")
    print(f"  Mean Average Precision MAP: {jaccard_results[query_id]['MAP']:.4f}")
    print(f"  nDCG@{k}: {jaccard_results[query_id]['nDCG']:.4f}")


Consulta 1: Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios
Métricas para la Similitud Jaccard:
  Precisión en el top-k Prec@10: 1.0000
  Recall: 1.0000
  F1-score: 1.0000
  Mean Average Precision MAP: 1.0000
  nDCG@10: 1.0000

Consulta 2: Actividades extracurriculares y bienestar emocional en el campus universitario
Métricas para la Similitud Jaccard:
  Precisión en el top-k Prec@10: 0.2000
  Recall: 1.0000
  F1-score: 0.3333
  Mean Average Precision MAP: 1.0000
  nDCG@10: 1.0000

Consulta 3: Estrategias universitarias para reducir el estrés en estudiantes
Métricas para la Similitud Jaccard:
  Precisión en el top-k Prec@10: 0.1000
  Recall: 1.0000
  F1-score: 0.1818
  Mean Average Precision MAP: 1.0000
  nDCG@10: 1.0000


 ### Análisis y Comparación:

- Discutir cuál sistema es más efectivo y por qué.

 ### Comparación entre consultas:

- **Consulta 1:** Jaccard logra una precisión de 1.0 frente al 0.9 de Coseno, lo que sugiere que Jaccard es ligeramente mejor en esta consulta.
- **Consulta 2:** Jaccard supera a Coseno en esta consulta, con una Prec@10 de 0.2 frente a 0.1 en Coseno, y un F1-score más alto con un valor de 0.33 frente a 0.18 en Coseno.
- **Consulta 3:** Ambas técnicas arrojan resultados idénticos.

 ### Conclusión:

La técnica de Similitud Jaccard es ligeramente más efectiva que Coseno para estas consultas, ya que obtuvo mejor precisión y F1-score en la Consulta 1 y significativamente mejor en la Consulta 2, sin perder recall en ningún caso. Esto sugiere que Jaccard es más precisa al identificar documentos relevantes.